In [5]:
import numpy as np
import pickle

with open("txt_data", "rb") as fp:   #Pickling
    data = pickle.load(fp)
with open("txt_labels", "rb") as fp:   #Pickling
    labels = pickle.load(fp)

print(len(data), len(labels))

31702 31702


In [6]:
from sklearn.model_selection import train_test_split 

x_train, x_test, y_train, y_test = train_test_split(data,labels, test_size = 0.30, shuffle=True, random_state = 42)
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.10, shuffle=True, random_state = 33)

print('split done.')

split done.


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=500, use_idf=True, stop_words='english')
vectorizer.fit(data)
x_train = vectorizer.transform(x_train).toarray()
x_test = vectorizer.transform(x_test).toarray()
x_valid = vectorizer.transform(x_valid).toarray()
print('> tfidVectorization done.')

> tfidVectorization done.


In [8]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report

params = {
    'n_estimators':[5, 10, 20],
    'learning_rate':[0.001, 0.01, 1],
    'base_estimator':[DecisionTreeClassifier(max_depth=5), DecisionTreeClassifier(max_depth=10), 
                      DecisionTreeClassifier(max_depth=20), DecisionTreeClassifier(max_depth=30),
                      DecisionTreeClassifier(max_depth=50)]
}

clf = GridSearchCV(AdaBoostClassifier(), param_grid=params, n_jobs=-1, cv=2, verbose=10)

clf.fit(x_train, y_train)

clf = clf.best_estimator_
print()
print('Best params: ',clf)

y_pred = clf.predict(x_test)

print()
dt = pd.DataFrame(confusion_matrix(y_test, y_pred))
display(dt)
print(classification_report(y_test, y_pred, zero_division=0))

Fitting 2 folds for each of 45 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   16.6s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   30.3s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:  5.7min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:  7.5min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed: 10.6min
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed: 13.7min finished



Best params:  AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                         class_weight=None,
                                                         criterion='gini',
                                                         max_depth=50,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                         min_samples_split=2,
                                                         min_weight_fraction_leaf=0.0,
                                                         presort='deprecated',
          

,0,1,2,3,4,5,6,7,8
0,809,32,27,31,64,13,12,56,16
1,25,1013,35,29,58,3,19,32,21
2,32,35,240,18,55,7,7,20,16
3,38,41,20,186,61,11,10,25,16
4,47,57,52,46,852,21,11,38,29
5,21,16,3,9,50,321,10,17,6
6,8,6,2,1,6,0,1087,139,152
7,11,12,3,5,21,2,144,1110,419
8,10,7,2,2,7,4,170,431,1011


              precision    recall  f1-score   support

           0       0.81      0.76      0.79      1060
           1       0.83      0.82      0.83      1235
           2       0.62      0.56      0.59       430
           3       0.57      0.46      0.51       408
           4       0.73      0.74      0.73      1153
           5       0.84      0.71      0.77       453
           6       0.74      0.78      0.76      1401
           7       0.59      0.64      0.62      1727
           8       0.60      0.61      0.61      1644

    accuracy                           0.70      9511
   macro avg       0.70      0.68      0.69      9511
weighted avg       0.70      0.70      0.70      9511



In [9]:
import pickle

with open('CLF_adaboost','wb') as f:
    pickle.dump(clf,f)